## Dinamiška ir interaktyvi duomenų vizualizacija, remiantis duomenimis paskolos pagal rezidentiškumą
Plačiau: https://www.lb.lt/lt/m_statistika/t-paskolos-pagal-rezidentiskuma

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'browser'

## 1. Duomenų įkėlimas, jų peržiūra ir sutvarkymas

Pirma įkeliame duomenis patikriname ar yra tuščių ar pasikartojančių duomenų. 

In [2]:
# Load the dataset
file_path = 'paskolos-pagal-rezidentiskuma.csv' 
data = pd.read_csv(file_path, delimiter=';')

print(data.columns)

# Check for duplicate rows
duplicates = data.duplicated().sum()
print(f'Total duplicate rows: {duplicates}')

# Check for missing values
missing_values = data.isnull().sum().sum()
print(f'Total missing (empty) values: {missing_values}')

# Show detailed missing values for each column 
print("Missing values per column:")
print(data.isnull().sum())
print(data.head())


Index(['code', 'en_long_title', 'lt_long_title', 'date', 'value', 'power'], dtype='object')
Total duplicate rows: 0
Total missing (empty) values: 0
Missing values per column:
code             0
en_long_title    0
lt_long_title    0
date             0
value            0
power            0
dtype: int64
                                       code  \
0  BPS.M.A20.A.1.00.000.000.LT.N.PAB__.E.SR   
1  BPS.M.A20.A.1.00.000.000.LT.N.PAB__.E.SR   
2  BPS.M.A20.A.1.00.000.000.LT.N.PAB__.E.SR   
3  BPS.M.A20.A.1.00.000.000.LT.N.PAB__.E.SR   
4  BPS.M.A20.A.1.00.000.000.LT.N.PAB__.E.SR   

                                       en_long_title  \
0  Monetary financial institutions balance sheet ...   
1  Monetary financial institutions balance sheet ...   
2  Monetary financial institutions balance sheet ...   
3  Monetary financial institutions balance sheet ...   
4  Monetary financial institutions balance sheet ...   

                                       lt_long_title     date   value    power

Duomenų rinkinys yra tvarkingai suformuotas (nėra trūkstamų duomenų ar dublikatų) ir susideda iš šių stulpelių:

code: Unikalus identifikatorius, priskirtas kiekvienai duomenų rinkinio eilutei.

en_long_title: Rodiklis, kuris nurodo, kokią informaciją ar duomenis šis rodiklis reprezentuoja anglų kalba.

lt_long_title: Rodiklis, kuris pateikia tokią pačią informaciją kaip ir „en_long_title“, tik lietuviškai.

date: Paskolos data, pateikta metai-mėnuo formatu (YYYY-MM).

value: Paskolos suma.

power: Skaitinė galia arba faktorius, kuris papildo „value“ reikšmę.


Toliau konvertuojame 'date' į datetime formatą, išgauname paskolų kategorijas iš 'lt_long_title' stulpelio, kur kiekvienos paskolos kategorija nustatoma pagal pirmąjį aprašymo žodį. Toliau sukuriame interaktyvią stulpelinę diagramą, vaizduojančią paskolų pasiskirstymą per laiką pagal šias kategorijas. Taip pat pateikiame interaktyvią linijinę diagramą, kuri leidžia analizuoti paskolų tendencijas skirtingose kategorijose bėgant laikui. 

In [10]:
data['date'] = pd.to_datetime(data['date'], format='%Y-%m')

data['adjusted_value'] = data['value'] * data['power']

data['category'] = data['lt_long_title'].apply(lambda x: x.split(' ')[0])

fig1 = px.bar(data, x="date", y="adjusted_value", color="category",
              title="Loan Distribution by Category over Time",
              labels={"adjusted_value": "Total Loan Amount (€)", "date": "Date", "category": "Category"})

fig2 = px.line(data, x="date", y="adjusted_value", color="category", markers=True,
               title="Loan Trends by Category",
               labels={"adjusted_value": "Total Loan Amount (€)", "date": "Date", "category": "Category"})

dropdown_buttons = [
    {'label': 'All Categories', 'method': 'update', 'args': [{'visible': [True]*len(data['category'].unique())}, {'title': 'Loan Distribution by All Categories'}]},
]


for category in data['category'].unique():
    dropdown_buttons.append(
        {'label': category,
         'method': 'update',
         'args': [{'visible': [True if cat == category else False for cat in data['category']]},
                  {'title': f'Loan Distribution for {category}'}]}
    )


fig1.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
        }
    ]
)

fig2.update_layout(
    updatemenus=[
        {
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
        }
    ]
)



fig1.show()
fig2.show()


In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = 'browser'

file_path = 'paskolos-pagal-rezidentiskuma.csv'
data = pd.read_csv(file_path, delimiter=';')

data['date'] = pd.to_datetime(data['date'], format='%Y-%m')


data['adjusted_value'] = data['value'] * data['power']


data['category'] = data['lt_long_title'].apply(lambda x: x.split(' ')[0])

data['category'] = data['category'].replace({'Pinigų': 'Value'})  


fig = go.Figure()

categories = data['category'].unique()


for category in categories:
    category_data = data[data['category'] == category]
    
    # Bar chart trace
    fig.add_trace(
        go.Bar(
            x=category_data['date'],
            y=category_data['adjusted_value'],
            name=f'{category} (Bar)',
            visible=True,  
        )
    )
    
    # Line chart trace
    fig.add_trace(
        go.Scatter(
            x=category_data['date'],
            y=category_data['adjusted_value'],
            mode='lines+markers',
            name=f'{category} (Line)',
            visible=False  
        )
    )


dropdown_buttons = [
    {
        'label': 'Bar Chart',
        'method': 'update',
        'args': [
            {'visible': [True if i % 2 == 0 else False for i in range(2 * len(categories))]},
            {'title': 'Loan Distribution by Category - Bar Chart'}
        ]
    },
    {
        'label': 'Line Chart',
        'method': 'update',
        'args': [
            {'visible': [True if i % 2 == 1 else False for i in range(2 * len(categories))]},
            {'title': 'Loan Distribution by Category - Line Chart'}
        ]
    },
    {
        'label': 'Both',
        'method': 'update',
        'args': [
            {'visible': [True] * (2 * len(categories))},  
            {'title': 'Loan Distribution by Category - Both Charts'}
        ]
    }
]


fig.update_layout(
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
    }],
    title="Loan Distribution by Category Over Time",
    xaxis_title="Date",
    yaxis_title="Total Loan Amount (€)"
)


fig.show()
